In [14]:
import json
import os

In [15]:
# 从负载的trace文件里读取trace，按结束时间排序
def getTrace():
    dir = './out/trace/'
    files = os.listdir(dir)
    raw_data = []
    for i in files:
        raw_data += json.loads(open(dir+i,'r',encoding='utf8').read())

    data = []
    for i in raw_data:
        _ = {}
        _['opID'] = i['operationID']
        _['startTime'] = i['startTimestamp']
        _['endTime'] = i['finishTimestamp']
        data.append(_)
    data = sorted(data, key = lambda x: x['endTime'])

    return data

In [16]:
# 根据依赖构建依赖的邻接表，这个依赖文件来自 src/main/java/ana/output/OutputStructure.java
def getAdj():
    adj = {}
    for l in open('./out/debug.json','r',encoding='utf8').readlines():
        if ':' not in l or len(l.split(',')) != 5:
            continue
        
        _ = l[:].split(':')[-1].split('*')
        if _[1] in adj and _[0] not in adj[_[1]]:
            adj[_[1]].add(_[0])
        else:
            adj[_[1]] = {_[0]}
    return adj

In [17]:
trace = getTrace()
adj = getAdj()
seq = []
cnt = 0
while len(trace) > 0:

    # getCandidateOperationId
    endT = trace[0]['endTime']
    candidate = []
    for i in range(len(trace)):
        if trace[i]['startTime'] < endT:
            candidate.append(trace[i])

    # getNotDependOperationId
    ret = set()
    for i in candidate:
        id = i['opID']
        if id in adj:
            if len(adj[id]) == 0:
                ret.add(id)
        else:
            ret.add(id)
    # removeDependency
    for i in adj:
        adj[i] -= ret

    
    seq.append(list(ret))
    
    # removeOperationTraces
    del_item = []
    for i in range(len(trace)):
        if trace[i]['opID'] in ret:
            del_item.append(trace[i])
    
    for i in del_item:
        trace.remove(i)



In [18]:
json.dump(seq,open('seq.json','w',encoding='utf8'))